In [1]:
import sympy
from sympy import *
etav, etae, av, ae = symbols('eta_v, eta_e, av, ae')
Exx, Exy = symbols('Exx_eff, Exy_eff')
Txx0, Txy0 = symbols('Txx0, Txy0')
sympy.__version__

'1.11.1'

In [2]:
# Make Newton iterations for VE power law
eta_e, a_v, a_e, eta_ve, eta_pwl = symbols('eta_e, a_v, a_e, eta_ve, eta_pwl')
Exx_eff, Eyy_eff, Exy_eff, C, B, npwl = symbols('Exx_eff, Eyy_eff, Exy_eff, C, B, npwl')

a_ve      = sqrt((eta_e*a_v**2 + eta_pwl*a_e**2)/(eta_e + eta_pwl)) 
eta_ve    = (1/eta_pwl + 1/eta_e)**-1
Txx       = 2*eta_ve * Exx_eff
Tyy       = 2*eta_ve * Eyy_eff
Txy       = 2*eta_ve * Exy_eff / (a_ve)**2
Tzz       = -Txx - Tyy 
T2_v      = 0.5*(Txx**2 + Tyy**2 + Tzz**2) + Txy**2*a_v**2
Eii_pwl   = C * sqrt(T2_v) **(npwl)
eta_pwl1  = B**(-1/npwl) * Eii_pwl**((1-npwl)/npwl)
f         = 1 - eta_pwl1/eta_pwl


In [3]:
display(f)

1 - (C*(2*sqrt(0.5*Exx_eff**2/(1/eta_pwl + 1/eta_e)**2 + Exy_eff**2*a_v**2*(eta_e + eta_pwl)**2/((1/eta_pwl + 1/eta_e)**2*(a_e**2*eta_pwl + a_v**2*eta_e)**2) + 0.5*Eyy_eff**2/(1/eta_pwl + 1/eta_e)**2 + 0.125*(-2*Exx_eff/(1/eta_pwl + 1/eta_e) - 2*Eyy_eff/(1/eta_pwl + 1/eta_e))**2))**npwl)**((1 - npwl)/npwl)/(B**(1/npwl)*eta_pwl)

In [4]:
dfde = f.diff(eta_pwl).subs(f,'f').subs(eta_pwl1,'eta_pwl').subs(Eii_pwl,'Eii_pwl').subs(T2_v,'T2_v').subs(Txx, 'Txx').subs(Tyy, 'Tyy').subs(Txy, 'Txy').subs(eta_ve,'eta_ve')
dfde = dfde.subs(Txx/2/eta_pwl, 'Exx_pwl')

In [5]:
print( 'dfde = ' + sympy.julia_code(dfde) + ';' ) 

dfde = -(1 - npwl) .* (0.5 * Exx_eff .^ 2 .* eta_ve .^ 3 ./ eta_pwl .^ 2 - Exy_eff .^ 2 .* a_e .^ 2 .* a_v .^ 2 .* eta_ve .^ 2 .* (eta_e + eta_pwl) .^ 2 ./ (a_e .^ 2 .* eta_pwl + a_v .^ 2 .* eta_e) .^ 3 + Exy_eff .^ 2 .* a_v .^ 2 .* eta_ve .^ 2 .* (eta_e + eta_pwl) ./ (a_e .^ 2 .* eta_pwl + a_v .^ 2 .* eta_e) .^ 2 + Exy_eff .^ 2 .* a_v .^ 2 .* eta_ve .^ 3 .* (eta_e + eta_pwl) .^ 2 ./ (eta_pwl .^ 2 .* (a_e .^ 2 .* eta_pwl + a_v .^ 2 .* eta_e) .^ 2) + 0.5 * Eyy_eff .^ 2 .* eta_ve .^ 3 ./ eta_pwl .^ 2 + 0.0625 * (-Txx - Tyy) .* (-2 * Txx .* eta_ve ./ eta_pwl .^ 2 - 2 * Tyy .* eta_ve ./ eta_pwl .^ 2)) ./ (0.5 * Exx_eff .^ 2 .* eta_ve .^ 2 + Exy_eff .^ 2 .* a_v .^ 2 .* eta_ve .^ 2 .* (eta_e + eta_pwl) .^ 2 ./ (a_e .^ 2 .* eta_pwl + a_v .^ 2 .* eta_e) .^ 2 + 0.5 * Eyy_eff .^ 2 .* eta_ve .^ 2 + 0.125 * (-Txx - Tyy) .^ 2) + 1 ./ eta_pwl;


In [17]:
# Make Newton ietrations for combined creep using normal and shear effective viscosities
eta_e, a_v, a_e, eta_ve, eta_pwl = symbols('eta_e, a_v, a_e, eta_ve, eta_pwl')
Exx_eff, Eyy_eff, Exy_eff, C, B, npwl = symbols('Exx_eff, Eyy_eff, Exy_eff, C, B, npwl')
eta_ve_n, eta_ve_s, eta_cst = symbols('eta_ve_n, eta_ve_s, eta_cst')

Txx       = 2*eta_ve_n * Exx_eff
Tyy       = 2*eta_ve_n * Eyy_eff
Txy       = 2*eta_ve_s * Exy_eff
Tzz       = -Txx - Tyy 

Ezz_eff   = -Exx_eff -Eyy_eff

Y2_v      = Rational(1,2)*(Txx**2 + Tyy**2 + Tzz**2) + Txy**2*a_v**2
Eii_pwl   = C * sqrt(Y2_v) **(npwl)
eta_pwl   = B**(-1/npwl) * Eii_pwl**((1-npwl)/npwl)
# eta_pwl   = B * Eii_pwl**((1-npwl)/npwl)

Wxx       = 2*( Txx*Exx_eff + Tyy*Eyy_eff + Tzz*Ezz_eff)
Wxy       = 2*  Txy*Exy_eff

fxx       = 1 -  eta_ve_n/ eta_e         - eta_ve_n/ eta_pwl         - 0*eta_ve_n/ eta_cst         
fxy       = 1 -  eta_ve_s/(eta_e/a_e**2) - eta_ve_s/(eta_pwl/a_v**2) - 0*eta_ve_s/(eta_cst/a_v**2) 

ieta_pwl = 1/ eta_pwl.subs(Eii_pwl, 'Eii_pwl').subs(Y2_v,'Y2_v')
dfxxde_n = fxx.diff(eta_ve_n).subs(Eii_pwl, 'Eii_pwl').subs(Y2_v,'Y2_v').subs(ieta_pwl,'ieta_pwl').subs(Wxx, 'Wxx').subs(Wxy, 'Wxy')
dfxxde_s = fxx.diff(eta_ve_s).subs(Eii_pwl, 'Eii_pwl').subs(Y2_v,'Y2_v').subs(ieta_pwl,'ieta_pwl').subs(Wxx, 'Wxx').subs(Wxy, 'Wxy')
dfxyde_n = fxy.diff(eta_ve_n).subs(Eii_pwl, 'Eii_pwl').subs(Y2_v,'Y2_v').subs(ieta_pwl,'ieta_pwl').subs(Wxx, 'Wxx').subs(Wxy, 'Wxy')
dfxyde_s = fxy.diff(eta_ve_s).subs(Eii_pwl, 'Eii_pwl').subs(Y2_v,'Y2_v').subs(ieta_pwl,'ieta_pwl').subs(Wxx, 'Wxx').subs(Wxy, 'Wxy')

print( 'dfxxde_n = ' + julia_code(dfxxde_n) + ';' ) 
print( 'dfxxde_s = ' + julia_code(dfxxde_s) + ';' ) 
print( 'dfxyde_n = ' + julia_code(dfxyde_n) + ';' ) 
print( 'dfxyde_s = ' + julia_code(dfxyde_s) + ';' ) 
print( )
print( 'dfxxde_n = ' + ccode(dfxxde_n) + ';' ) 
print( 'dfxxde_s = ' + ccode(dfxxde_s) + ';' ) 
print( 'dfxyde_n = ' + ccode(dfxyde_n) + ';' ) 
print( 'dfxyde_s = ' + ccode(dfxyde_s) + ';' ) 

print(fxx.diff(eta_ve_n))

dfxxde_n = -ieta_pwl - 1 ./ eta_e + eta_ve_n .* ieta_pwl .* (1 - npwl) .* (4 * Exx_eff .^ 2 .* eta_ve_n + 4 * Eyy_eff .^ 2 .* eta_ve_n + (-4 * Exx_eff - 4 * Eyy_eff) .* (-2 * Exx_eff .* eta_ve_n - 2 * Eyy_eff .* eta_ve_n) / 2) ./ (2 * Y2_v);
dfxxde_s = Wxy .* a_v .^ 2 .* eta_ve_n .* ieta_pwl .* (1 - npwl) ./ Y2_v;
dfxyde_n = a_v .^ 2 .* eta_ve_s .* ieta_pwl .* (1 - npwl) .* (4 * Exx_eff .^ 2 .* eta_ve_n + 4 * Eyy_eff .^ 2 .* eta_ve_n + (-4 * Exx_eff - 4 * Eyy_eff) .* (-2 * Exx_eff .* eta_ve_n - 2 * Eyy_eff .* eta_ve_n) / 2) ./ (2 * Y2_v);
dfxyde_s = Wxy .* a_v .^ 4 .* eta_ve_s .* ieta_pwl .* (1 - npwl) ./ Y2_v - a_e .^ 2 ./ eta_e - a_v .^ 2 .* ieta_pwl;

dfxxde_n = -ieta_pwl - 1/eta_e + (1.0/2.0)*eta_ve_n*ieta_pwl*(1 - npwl)*(4*pow(Exx_eff, 2)*eta_ve_n + 4*pow(Eyy_eff, 2)*eta_ve_n + (1.0/2.0)*(-4*Exx_eff - 4*Eyy_eff)*(-2*Exx_eff*eta_ve_n - 2*Eyy_eff*eta_ve_n))/Y2_v;
dfxxde_s = Wxy*pow(a_v, 2)*eta_ve_n*ieta_pwl*(1 - npwl)/Y2_v;
dfxyde_n = (1.0/2.0)*pow(a_v, 2)*eta_ve_s*ieta_pwl*(1 - npw

In [18]:

fff = 4*Exx_eff**2*eta_ve_n + 4*Eyy_eff**2*eta_ve_n + (-4*Exx_eff - 4*Eyy_eff)*(-2*Exx_eff*eta_ve_n - 2*Eyy_eff*eta_ve_n)/2 - Wxx
fff.simplify()

0

In [8]:
dfxxde_n = -ieta_pwl - 1 ./ eta_e + eta_ve_n .* ieta_pwl .* (1 - npwl) .* (4 * Exx_eff .^ 2 .* eta_ve_n + 4 * Eyy_eff .^ 2 .* eta_ve_n + (-4 * Exx_eff - 4 * Eyy_eff) .* (-2 * Exx_eff .* eta_ve_n - 2 * Eyy_eff .* eta_ve_n) / 2) ./ (2 * Y2_v);
dfxxde_s = Wxy .* a_v .^ 2 .* eta_ve_n .* ieta_pwl .* (1 - npwl) ./ Y2_v;

dfxxde_n = -ieta_pwl - 1 ./ eta_e + eta_ve_n .* ieta_pwl .* (1 - npwl) .* (4 * Exx_eff .^ 2 .* eta_ve_n + 4 * Eyy_eff .^ 2 .* eta_ve_n + (-4 * Exx_eff - 4 * Eyy_eff) .* (-2 * Exx_eff .* eta_ve_n - 2 * Eyy_eff .* eta_ve_n) / 2) ./ (2 * Y2_v);
dfxxde_s = Wxy .* a_v .^ 2 .* eta_ve_n .* ieta_pwl .* (1 - npwl) ./ Y2_v;

(C*(2*Exx_eff**2*eta_ve_n**2 + 4*Exy_eff**2*a_v**2*eta_ve_s**2 + 2*Eyy_eff**2*eta_ve_n**2 + (-2*Exx_eff*eta_ve_n - 2*Eyy_eff*eta_ve_n)**2/2)**(npwl/2))**((1 - npwl)/npwl)/B**(1/npwl)

In [9]:
(Txx**2 + Tyy**2 + Tzz**2)

4*Exx_eff**2*eta_ve_n**2 + 4*Eyy_eff**2*eta_ve_n**2 + (-2*Exx_eff*eta_ve_n - 2*Eyy_eff*eta_ve_n)**2

In [10]:
twoY2 = 2*Y2_v